In [1]:
from sqlalchemy import create_engine
import pandas as pd
from collections import Counter
import re

db_string = 'postgresql://localhost/postgres'

db = create_engine(db_string)

In [2]:
sql = '''SELECT * 
         FROM artworks'''

In [3]:
result_set = pd.read_sql(sql, db)

In [4]:
artworks_df = pd.DataFrame(result_set)

In [5]:
artworks_df.head()

,title,artist,artistbio,nationality,begindate,enddate,gender,date,medium,dimensions,...,circumference,diameter,height,length,weight,width,seatheight,duration,depth,constituentid
0,Exposition - January 1898,Cause,(French),(French),(0),(0),(),1898,Poster,None,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,1040
1,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",...,NaN,NaN,49.0,NaN,NaN,169.0,None,NaN,None,6210
2,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,"16 x 11 3/4"" (40.6 x 29.8 cm)",...,NaN,NaN,41.0,NaN,NaN,30.0,None,NaN,None,7470
3,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, pen, color pencil, ink, and gouache ...","13 1/2 x 12 1/2"" (34.3 x 31.8 cm)",...,NaN,NaN,34.0,NaN,NaN,32.0,None,NaN,None,7605
4,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),1980,Photographic reproduction with colored synthet...,"20 x 20"" (50.8 x 50.8 cm)",...,NaN,NaN,51.0,NaN,NaN,51.0,None,NaN,None,7056


In [ ]:
artworks_df.info()

## Cleaning Up Rows

Clean up rows:
- artistbio
- nationality
- begindate
- enddate
- gender
- date

In [7]:
# artistbio
# if-else in lambda statement allows me to replace NoneTypes with ''
artworks_df['artistbio'] = artworks_df['artistbio'].apply(lambda x: (x.replace('(', '').replace(')', '')) if x is not None else '')

In [8]:
# nationality
artworks_df['nationality'] = artworks_df['nationality'].apply(lambda x: (x.replace('(', '').replace(')', '')) if x is not None else '')

In [9]:
# begindate

artworks_df['begindate'] = artworks_df['begindate'].apply(lambda x: x.replace('(', '').replace(')', '').replace(' ', ',') if x is not None else '')

# can have multiple begindates, will take only the first four numbers as the begindate. 
# If first character is 0 and string less than 4, then 0. 
# Elif first character is 0, but string greater than 4, skip the 0 and take the next 4 numbers.
# Else: first 4 numbers.

def clean_beginend_date(string):
    if ',' in string:
        new_list = string.split(',')
        int_list = [int(i) for i in new_list]
        return max(int_list)
    elif string is None or string == '':
        return None
    else:
        return int(string)
    
artworks_df['begindate'] = artworks_df['begindate'].apply(clean_beginend_date)

In [10]:
# enddate

artworks_df['enddate'] = artworks_df['enddate'].apply(lambda x: x.replace('(', '').replace(')', '').replace(' ', ',') if x is not None else '')

# can have multiple enddates. We will select the most recent enddate if there are multiple dates applied.

artworks_df['enddate'] = artworks_df['enddate'].apply(clean_beginend_date)

In [11]:
# gender
# there can be multiple genders for one artwork (collaboration)

artworks_df['gender'] = artworks_df['gender'].apply(lambda x: x.replace('(', '').replace(')', '').replace(' ', ',') if x is not None else '')

# to better assign genders, if there were more of one gender involved in an artwork, I assigned the majority gender as the main gender. 
def gender_assign(string):
    if ',' in string:
        new_list = string.split(',')
        new_list = [i.capitalize() for i in new_list]
        counter = Counter()
        for each in new_list:
            if each == '':
                continue
            else:
                counter[each] += 1
                
        if len(counter) > 1:
            if counter.most_common()[0][1] == counter.most_common()[1][1]:
                return 'Male/Female'
            else:
                return str(counter.most_common()[0][0])
    else:
        return string.capitalize()
      
    
artworks_df['gender'] = artworks_df['gender'].apply(gender_assign)

In [12]:
# date
# first four characters if > 4 
# Else: return string
# one of the dates has a .1-3 1987; .4 1990 format.

artworks_df['date'] = artworks_df['date'].fillna('')

def clean_dates(string):
    
    if string == '' or string == None:
        return string
    
    
    if string[0].isnumeric() and string[1].isnumeric() and string[2].isnumeric():
        new_string = ''
        for each in string[:4]:
            new_string += each
        return new_string
    
    if string[0].isnumeric() and string[1].isnumeric():
        new_string = ''
        for each in string[:-5:-1]:
            new_string += each
        return new_string[::-1]
        
    if string[0] == '.':
        cleaned = re.sub("[^0-9]", "", string)
        new_string = ''
        for each in cleaned[2:6]:
            new_string += each
        return new_string
    
    if 'dated' in string and ';' in string and ',' in string: # we will take the dates that come after 'dated'
        cleaned = re.sub("[^0-9]", "", string)
        new_string = ''
        for each in cleaned[:-5:-1]:
            new_string += each
        return new_string[::-1]
        
    if string[0] == 'c' and string[1] == '.':
        cleaned = re.sub("[^0-9]", "", string)
        new_string = ''
        for each in cleaned[:4]:
            new_string += each
        return new_string
    
    if string[0].isalpha():
        cleaned = re.sub("[^0-9]", "", string)
        new_string = ''
        for each in cleaned[:-5:-1]:
            new_string += each
        return new_string[::-1]

artworks_df['date'] = artworks_df['date'].apply(clean_dates)

In [ ]:
# dimensions
# og dimension column has inches and centimeters. We only want to record the inches. 
# After further investigation, there are many different ways the MoMA dataset is handling the dimensions. Will leave as a string for now.
'''
from fractions import Fraction


artworks_df['dimensions'] = artworks_df['dimensions'].fillna('')

def clean_dimensions(string):
    if string == '' or string == None:
        return string
    
    elif string[0].isnumeric():
        split_string = string.replace('"', '').split('(')[0].split(' ')
        split_string = [float(Fraction(i)) for i in split_string if i != '' and i != 'x']
    
        if len(split_string) == 4:
            first = split_string[0] + split_string[1]
            second = split_string[2] + split_string[3]
            return first*second
        elif len(split_string) == 3:
            first = split_string[0]
            if split_string[1] < 1:
                first = first + split_string[1]
                second = split_string[2]
            else:
                second = split_string[1] + split_string[2]
            return first*second
        elif len(split_string) == 2:
            return split_string[0]*split_string[1]
        
    elif string[0] == 'E':
        cleaned = string.replace('"', '').split('(')[0].split(' ')
'''        

In [13]:
artworks_df['dimensions'] = artworks_df['dimensions'].fillna('')

In [15]:
artworks_df.to_sql('artworks_clean', db, if_exists='append', index = False)